In [1]:
import pandas as pd

In [3]:
df = pd.read_parquet('../../hces-2023-24/data/sec-1-HCQ.parquet')  # , usecols=['sector', 'state_name', 'district_name'])
df = df[['sector', 'nss_region']]
df.head()

,sector,nss_region
hhid,,
46667201,rural,12
46667301,rural,12
46667302,rural,12
46667303,rural,12
46667304,rural,12


In [4]:
xdf = pd.read_parquet('../../hces-2023-24/data/sec-4-HCQ.parquet')
xdf = xdf[['hh_size', 'max_income_from', 'hoh_religion', 'caste']]
xdf.head()

,hh_size,max_income_from,hoh_religion,caste
hhid,,,,
22300101,4,self-employment,Hinduism,other backward class (OBC)
22300201,4,self-employment,Hinduism,other backward class (OBC)
22300202,2,None,Hinduism,other backward class (OBC)
22300203,2,None,Hinduism,other backward class (OBC)
22300204,2,regular wage/salary earning,Christianity,other backward class (OBC)


In [5]:
df = pd.concat([df, xdf], axis=1, verify_integrity=True)

In [6]:
df['family_size'] = pd.cut(df['hh_size'], [0, 5, df['hh_size'].max()], labels=['nuclear', 'large'], right=True, include_lowest=True)

In [7]:
hdf = pd.read_parquet('../../hces-2023-24/data/sec-3-HCQ.parquet')
hdf.head()

rel_to_hoh  gender  age     marital_status  \
hhid     person_id                                                    
22300101 3          unmarried child    male   23      never married   
         2                   spouse  female   45  currently married   
         4          unmarried child  female   24      never married   
         1                     self    male   56  currently married   
22300201 2                   spouse  female   44  currently married   

                                  edu_level  edu_years used_internet  \
hhid     person_id                                                     
22300101 3                         graduate       18.0           yes   
         2                        secondary       10.0           yes   
         4          post graduate and above       20.0           yes   
         1                         graduate       15.0           yes   
22300201 2             upper primary/middle        8.0            no   

                    days_outside_home  meals_daily  meals_school  \
hhid     person_id                                                 
22300101 3                        0.0          3.0           NaN   
         2                        0.0          3.0           NaN   
         4                        0.0          3.0           NaN   
         1                        0.0          3.0           NaN   
22300201 2                        0.0          3.0           NaN   

                    meals_employer  meals_others  meals_payment  meals_home  \
hhid     person_id                                                            
22300101 3                     NaN           NaN            3.0        87.0   
         2                     NaN           3.0            NaN        87.0   
         4                     NaN           NaN            4.0        86.0   
         1                     NaN           NaN            3.0        87.0   
22300201 2                     NaN           NaN            NaN        90.0   

                    member_status  original_member  multiplier  
hhid     person_id                                              
22300101 3                      1              1.0       57436  
         2                      1              1.0       57436  
         4                      1              1.0       57436  
         1                      1              1.0       57436  
22300201 2                      1              1.0       27497

In [9]:
children = hdf['age'] <= 15
elderly = hdf['age'] >= 60

df['has_child'] = children.groupby(children.index.get_level_values(0)).any()
df['has_elderly'] = elderly.groupby(elderly.index.get_level_values(0)).any()

In [10]:
df = df[['sector', 'nss_region', 'max_income_from', 'hoh_religion', 'family_size', 'has_child', 'has_elderly', 'caste']]
df

,sector,nss_region,max_income_from,hoh_religion,family_size,has_child,has_elderly,caste
hhid,,,,,,,,
46667201,rural,12,self-employment,Hinduism,nuclear,False,False,others
46667301,rural,12,regular wage/salary earning,Hinduism,nuclear,False,True,others
46667302,rural,12,self-employment,Islam,nuclear,True,False,others
46667303,rural,12,regular wage/salary earning,Hinduism,nuclear,True,True,others
46667304,rural,12,None,Hinduism,nuclear,False,True,others
...,...,...,...,...,...,...,...,...
43277314,rural,101,casual labour,Hinduism,nuclear,True,False,scheduled caste (SC)
43277315,rural,101,casual labour,Hinduism,nuclear,True,False,scheduled caste (SC)
43277316,rural,101,casual labour,Hinduism,nuclear,True,False,other backward class (OBC)


In [12]:
df['cohort_id'] = df.groupby(df.columns.tolist(), dropna=False).ngroup()

In [13]:
df.head()

,sector,nss_region,max_income_from,hoh_religion,family_size,has_child,has_elderly,caste,cohort_id
hhid,,,,,,,,,
46667201,rural,12,self-employment,Hinduism,nuclear,False,False,others,160
46667301,rural,12,regular wage/salary earning,Hinduism,nuclear,False,True,others,116
46667302,rural,12,self-employment,Islam,nuclear,True,False,others,187
46667303,rural,12,regular wage/salary earning,Hinduism,nuclear,True,True,others,123
46667304,rural,12,None,Hinduism,nuclear,False,True,others,207


In [18]:
df.to_parquet('data/hh-cohorts.parquet')